In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct  5 17:38:16 2019

@author: unalunsal
https://www.eia.gov/totalenergy/data/browser/?tbl=T09.09#/?f=M
"""

import numpy as np
import pandas as pd
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure
#from bokeh.sampledata.stocks import AAPL
#from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
tf.random.set_seed(13)

In [3]:
# LSTM model inputs 
train_size = 530
vec_node = 20
step_ahead = 0
batch_size = 256
buffer_size = 10000
internal_eval = 200
epochs = 10

In [4]:
df = pd.read_csv(r'C:\Users\unalu\Desktop\eia\MER_T09_09.csv')
df['YYYYMM'] = df['YYYYMM'].astype(str).values # convert integer to string 

df = df.loc[~df['YYYYMM'].str.endswith('13'),:]  # drop the annual values 
 
df['YYYYMM'] = pd.to_datetime(df['YYYYMM'], format='%Y%m', errors='coerce').dropna() # convert string to datetime

df_clerdus = df.loc[df['MSN'] == 'CLERDUS',]

dates = df_clerdus['YYYYMM']

data_ts = df_clerdus['Value'].values.astype(np.float)

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []
  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)


lstm_pred = np.array([])
for i in range(train_size, len(data_ts)):   
    data_ts_mean = data_ts[:i].mean()
    data_ts_std = data_ts[:i].std()
    data_ts = (data_ts-data_ts_mean) / data_ts_std  

    x_train_uni, y_train_uni = univariate_data(data_ts, 0, train_size, vec_node, step_ahead)
    x_val_uni, y_val_uni = univariate_data(data_ts, train_size, None, vec_node, step_ahead)

    train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
    train_univariate = train_univariate.cache().shuffle(buffer_size).batch(batch_size).repeat()

    val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
    val_univariate = val_univariate.batch(batch_size).repeat()

    lstm_mod = tf.keras.models.Sequential([
                        tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
                        tf.keras.layers.Dense(1)])
    
    lstm_mod.compile(optimizer='adam', loss='mae')
    
    lstm_mod.fit(train_univariate, epochs=epochs,
                          steps_per_epoch=internal_eval,
                          validation_data=val_univariate, validation_steps=50)
    for x, y in val_univariate.take(3):
        forecast = lstm_mod.predict(x)[0].item() * data_ts_std + data_ts_mean
    lstm_pred = np.append(lstm_pred, forecast)

In [ ]:
forecast_ts = df_clerdus['Value'][:train_size].append(pd.Series(round(lstm_pred, 2)))